# Math  1376: Programming for Data Science
---

In [ ]:
import numpy as np  # We will use numpy in this lecture
import matplotlib.pyplot as plt
%matplotlib widget

from matplotlib.patches import Polygon 
from matplotlib.patches import Rectangle
from matplotlib.collections import PatchCollection

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Module 04: Some useful applications of Modules 01-03
---

In this module, we will now pull together material across our first three modules to solve some practical problems. 

You may find it useful to review the notebooks in those modules beforehand or just simply open some of the notebooks to have their contents available to review as necessary. 

While there are a seemingly endless number of practical problems we can attempt to solve with what we have learned so far, we will focus on three ubiquitous problems in the computational sciences.

- Root-finding. (The content of Part (a) of this module.)

- Numerical integration. (The content of Part (b) of this module.)

- Optimization. (This content is pursued in the assignments of this module.)


### A note about calculus concepts and interactive visualizations (i.e., widgets)
---

These topics are commonly studied as applications of calculus concepts. 
However, while we may make passing reference to certain calculus concepts, you do *not* need to know calculus to follow the narratives. (This, of course, is not to say that you should not seek to master calculus at some point.)
Our focus is on the *big picture ideas* and we use interactive graphics (powered by a widgets module) to help us explore these ideas.

## Learning Objectives for Part (b)

- Understand what integration means and how it arises in practical applications.


- Implement different types of numerical integration algorithms to various functions written either as `lambda` Python functions or more complicated user-defined functions.


- Create code that implements numerical integration algorithms of different types.


- Create annotations and interactive widgets to enhance visualizations of data.


### Some larger "in situ" learning objectives (i.e., learning that will occur by design of activities)

While we are going to explore how to implement some of these algorithms as activities below, our learning objectives go beyond simple correct implementation. We will also consider what it means to do the following:

- *compare* and *analyze* different algorithms developed for solving the same generic problem;

- use this comparison to *choose* the "right" algorithm for solving a *specific* problem;

- create a module (in the external activity) that encodes various algorithms and a *wrapper* function that automatically chooses which algorithm to apply based on the inputs.

Some of this is done in the notebook while other parts are left for homework.

## Notebook contents <a id='Contents'></a>

* <a href='#Integration'>Part (b): Numerical integration</a>

    * <a href='#concepts'>Part (b)(i): The basic concepts of integration</a>

    * <a href='#numerical-integration'>Part (b)(ii): Numerical integration algorithms</a>

        * <a href='#activity-rectangle-rules'>Activity: Rectangle rules</a>

        * <a href='#activity-MC'>Activity: Monte Carlo integration</a>

        * <a href='#activity-MC-darts'>Activity: Integration over the dart board</a>

    * <a href='#activity-summary'>Activity: Summary</a>

## Part (b): Numerical integration <a id='Integration'>
---

**Expected time to completion: 6-9 hours**    

## Part (b)(i): The basic concepts of integration <a id='concepts'>

<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the first recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell will embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('Fd58TZj6M2w', width=800, height=450)

### What is it and why should we care?

What *the integral* of a function means can depend a bit on context, which also helps to explain the *why* of an integral. 
Typically, it refers to transforming a function into a scalar quantity that describes some type of important aggregate behavior of the function over a set.
It is *kind of like* summing up the behavior of a function over a set in order to make important inferences. 
Some examples of what integrals mean in different contexts are given below. 

- In probability theory, the functions that quantify scalar outputs of an experiment are called random variables. 
The integrals of random variables weighted by their probability density functions give the expected value of the experiment.
Other standard statistical quantities such as variance also involve integrals.

- In engineering design and manufacturing processes, it is often important to compute the length of a curve, the area of a region, or the volume of an object (e.g. to determine the amount of resources/cost in constructing an object). Such quantities are given by integrals.

- In physics, integrals are used to determine important quantities like velocity (which is given as an integral of acceleration) and displacement (an integral of velocity). 

- In finance, integrals are sometimes used to determine options pricing.


- Many models of complex physical phenomena involve partial differential equations that are solved via numerical methods (e.g., finite element methods) that require computations of many integrals to construct accurate approximations. 


- In data science, statistical/machine learning, and any other data-driven discipline where proposed models are *fitted* to data, the *goodness of fit* of a model can usually be described in terms of an integral (or its discrete counterpart: summation). 

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Notation:*** <span>

There are a few different notational conventions, but here we will use the following: Let $f(x)$ be a function and $A\subset dom(f)$ (i.e., $A$ is some set of acceptable inputs taken from the domain of the function $f$), then the integral of $f$ over $A$ is denoted by

$$
  \large  \int_A f(x)
$$

### A standard motivating example

We are going to avoid any complicating calculus details.
But, to build intuition, we are going to consider some simple examples that also serve to make all of this less abstract and more concrete. 

<span style='background:rgba(255,0,255, 0.25); color:black'> ***General details:*** <span>

- Suppose $v(t)$ describes the velocity of an object moving either forward/backward on some path over the time interval $[t_0,t_f]$ (here $t_0$ denotes an initial time and $t_f$ denotes the final time).

- Suppose we are interested in how far along the path the object ultimately ends up relative to its starting position, which is given by

$$
  \large  \int_{[t_0,t_f]} v(t)
$$

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Examples with intuitive solutions:*** <span>

For simple functions of $v(t)$, we can easily conceptualize the problem and intuit the solution with little difficulty (i.e., without referring to anything from calculus). Consider the following scenarios:

- $v(t)=0$ (i.e., the object is not moving). 

    - Then, $\int_{[t_0,t_f]} v(t) = 0$, which means the final displacement of the object from its initial position is zero units of length. Not surprising. The object did not move.

    
- Suppose now that
    
$$
    v(t) = \begin{cases}
                5, & t_0<t<\frac{t_f+t_0}{2}, \\
                -5, & \frac{t_f+t_0}{2}<t<t_f,
            \end{cases}
$$
    
which simply means that the object is moving forward at a constant speed of 5 (ignoring units) for half the time and then moving at the same speed *but backwards* (i.e., in the other direction) the other half of time. 

  - Then, $\int_{[t_0,t_f]} v(t) = 0$ because the object just did a simple "round trip" back to where it started. 



- Suppose we have the same velocity as the previous example, but we instead we wanted to know the *total distance* traveled instead of just the final displacement from the starting position? Then, we would want to know $$\large \int_{[0,t_f]} \vert v(t) \vert$$ which means we want to integrate the absolute value of velocity. 
   <br><br>
   
   But, what is $\int_{[t_0,t_f]}\vert v(t) \vert$?
   <br><br>
   
   In this case, we know that this means we are integrating a function that is constant over all time (in this case a constant 5). If an object moves at a constant *speed* (speed is the absolute value of velocity), then we should be able to figure out the total distance it traveled. This is easier when considering units. Suppose $v(t)$ is described in miles per hour, which we rewrite as $\frac{\text{miles}}{\text{hour}}$. It sure seems like if we just multiplied by the total number of hours the object was moving, then we would get the total number of miles the object traveled because 
   
   $$
       \frac{\text{miles}}{\text{hour}}\text{hour} = \text{miles},
   $$
   
    i.e., the "hours cancel out."
   <br><br>
   
   So, in this example, $\int_{[t_0,t_f]} \vert v(t) \vert = 5(t_f-t_0)$.
   <br><br>
   
   Let's visualize these last two results.

In [ ]:
def v(t,t_f):
    n = len(t)
    vs = np.zeros(n)
    for i in range(n):
        if t[i]<t_f/2:
            vs[i] = 5
        else:
            vs[i] = -5
    return vs

In [ ]:
t_0 = 0
t_f = 10  # integral limits
t = np.linspace(t_0, t_f)

%matplotlib widget
fig, ax = plt.subplots(num=0)
ax.plot(t, np.abs(v(t,t_f)), 'r', linewidth=2)
ax.set_ylim(bottom=0)

# Make the shaded region associated with the integral
it = np.linspace(t_0, t_f)
iv = np.abs(v(it,t_f))
verts = [(t_0, 0), *zip(it, iv), (t_f, 0)]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
ax.add_patch(poly)

ax.text(0.5 * (t_0 + t_f), 2.5, r"$\int_{[t_0,t_f]} \vert v(t)\vert=5(t_f-t_0)$",
        horizontalalignment='center', fontsize=20)

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xticks((t_0, t_f))
ax.set_xticklabels(('$t_0$', '$t_f$'), fontsize=20);

In [ ]:
t_0 = 0
t_f = 10  # integral limits
t = np.linspace(t_0, t_f, 101)

%matplotlib widget
fig, ax = plt.subplots(num=1)
ax.plot(t, v(t,t_f), 'r', linewidth=2)

# Make the shaded region associated with the integral
it = np.linspace(t_0, t_f, 101)
iv = v(it,t_f)
verts = [(t_0, 0), *zip(it, iv), (t_f, 0)]
poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
ax.add_patch(poly)

ax.text(0.75 * (t_0 + t_f), 2.5, r"$\int_{[t_0,t_f]} v(t)=0$",
        horizontalalignment='center', fontsize=20)

ax.axhline(0, linewidth=1, c='k')  # plot $v=0$ line to more clearly demonstrate the positive/negative parts

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xticks((t_0, t_f))
ax.set_xticklabels(('$t_0$', '$t_f$'), fontsize=20);

## What do we see?

- It appears that these integrals are related to the *areas* of the shaded rectangles. 

    In particular, $\int_{[t_0,t_f]} |v(t)|$ is exactly the area of the single shaded rectangle.


- On the other hand, $\int_{[t_0,t_f]} v(t)$ is given by the area of the rectangle above the $t$-axis *minus* the area of the rectangle below the $t$-axis. They appear to *cancel* each other out. In fact, if we think of areas as being *signed* so that any area above a horizontal axis is positive and any area below the horizontal axis is negative, then we see that $\int_{[t_0,t_f]} v(t)$ is actually the sum of the *signed* areas. 


- These observations actually lead us to a useful conceptualization of integrals in terms of "sizes" of positive and negative regions of a function over a set. 

Below, we visualize what the sign of an integral is for a function in terms of what dominates: the positive or negative areas `quad` function to compute accurate approximations of the integral.

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Key takeaways about `quad`:*** </span>

- The `quad` function is within the `integrate` subpackage of `scipy`, and you should take at least 5-10 minutes to review some of the documentation https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html

- The `quad` function returns a tuple as an output (much like our `compute_bisection` function from the previous lecture notebook). 
The first component is the approximation of the integral. The second component is an approximation of the error in the integral. Therefore, we usually append a `[0]` to the end of it whenever we are just interested in the actual approximation of the integral (you should take note of this below).

- If you peruse the source code for the `quad` function, then you will see some nice docstrings, doctests, etc. https://github.com/scipy/scipy/blob/v1.5.3/scipy/integrate/quadpack.py#L49-L442

The `integrate` subpackage is itself rather interesting and useful to familiarize yourself with: https://docs.scipy.org/doc/scipy/reference/tutorial/integrate.html

In [ ]:
from scipy.integrate import quad

In [ ]:
def plot_integral(f, x_min, x_max, a, b, fignum=0):
    # Estimate the integral with quad function
    # The quad returns a tuple and the first component is
    # the estimate of the integral we are after
    int_ab = quad(f, a, b)[0] 
    
    #### EVERYTHING ELSE BELOW IS JUST PLOTTING
    x = np.linspace(x_min, x_max, 101)
    y = f(x)

    plt.figure(num=fignum)
    plt.clf()
    fig, ax = plt.subplots(num=fignum)
    ax.plot(x, y, 'r', linewidth=2)

    plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis
    
    # Make the shaded region
    ix = np.linspace(a, b, 101)
    iy = f(ix)
    # A * before the zip will "unpack" the tuples in the 
    # list of tuples created by zip
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)
    
    ax.set_title(r"$\int_a^b f(x)\mathrm{d}x \approx $ %3.2f" %int_ab, fontsize=10)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_xticks((a, b))
    ax.set_xticklabels(('$a$', '$b$'))
    plt.show()

In [ ]:
%reset -f out

%matplotlib widget
interact_manual(plot_integral, 
         f = widgets.fixed(lambda x: (x + 1) * (x - 5) * np.sin(3*x)),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(5),
         a = widgets.FloatSlider(value=2, min=-1, max=5, step=0.1),
         b = widgets.FloatSlider(value=4, min=-1, max=5, step=0.1),
         fignum = widgets.fixed(0));

**Creating an `Integration` class.**

<span style='background:rgba(0,255,255, 0.5); color:black'> Suggested activity:</span> 
- Add docstrings to the `Integration` class below.

- Add code comments to this class.

In [ ]:
class Integration(object):
    '''
    
    '''
    def __init__(self, f, a=None, b=None):
        '''
        
        '''
        self.f = f
        self.a = a
        self.b = b
        
        return
    
    def set_integral_limits(self, a, b):
        '''
        
        '''
        self.a = a
        self.b = b
        
        return
    
    
    def evaluate_integral(self, a=None, b=None):
        '''
        
        '''
        if (a is not None) and (b is not None):
            self.set_integral_limits(a, b)
            
        self.integral = quad(self.f, self.a, self.b)[0]
        
        return self.integral
    
    def plot_integral(self, x_min=None, x_max=None, N=101, fignum=0):
        '''
        
        '''
        if x_min is None:
            x_min = self.a
        
        if x_max is None:
            x_max = self.b
            
        x = np.linspace(x_min, x_max, N)
        y = self.f(x)

        plt.figure(num=fignum)
        plt.clf()
        fig, ax = plt.subplots(num=fignum)
        ax.plot(x, y, 'r', linewidth=2)

        plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis

        # Make the shaded region
        ix = np.linspace(self.a, self.b, N)
        iy = self.f(ix)
        # A * before the zip will "unpack" the tuples in the 
        # list of tuples created by zip
        verts = [(self.a, 0), *zip(ix, iy), (self.b, 0)]
        poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
        ax.add_patch(poly)

        ax.set_title(r"$\int_a^b f(x)\mathrm{d}x \approx $ %3.2f" %self.integral, fontsize=10)
            
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xticks((self.a, self.b))
        ax.set_xticklabels(('$a$', '$b$'))
        plt.show()
        
        return
        
    def evaluate_and_plot_integral(self, a=None, b=None, x_min=None, x_max=None, fignum=0):
        '''
        
        '''
        self.evaluate_integral(a=a, b=b)
        
        self.plot_integral(x_min=x_min, x_max=x_max, fignum=fignum)
        
        return

In [ ]:
f_integral = Integration(lambda x: (x + 1) * (x - 5) * np.sin(3*x))

In [ ]:
%reset -f out

%matplotlib widget
interact_manual(f_integral.evaluate_and_plot_integral, 
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(5),
         a = widgets.FloatSlider(value=2, min=-1, max=5, step=0.1),
         b = widgets.FloatSlider(value=4, min=-1, max=5, step=0.1),
         N = widgets.fixed(101), 
         fignum = widgets.fixed(0));

### Context is everything and these are just 1-dimensional examples

Above, we have just looked at functions with univariate inputs. Subsequently, we observed that the integral is related to the *signed* area between the curve defined by the function and the axis defined by the input.


This observation can be conceptually extended to functions with multivariate inputs. Suppose $f(x)$ is a real-valued function, but $x=(x_1,x_2)$ is a point in $\mathbb{R}^2$. Then, for a set $A\subset\mathbb{R}^2$, the integral $\int_A f(x)$ is related to the *signed volume* between the *surface* given by the graph of $f(x)$ over $A$ and the horizontal *plane* defined by the 2-dimensional input.

Generalizing the concept of area/volume is necessary to describe integrals in geometric terms when the inputs are three-dimensional or higher. 

We can pretty much stick with 1-dimensional examples to explore the typical numerical algorithms used to estimate integrals although some of the strengths and weaknesses of these algorithms are not apparent until we get to higher dimensions. Keep that in mind.

## Part (b)(ii): Numerical Integration <a id='numerical-integration'>

<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the first recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell will embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('GyWZKfm6y2A', width=800, height=450)

There are lots of algorithms to perform numerical integration, e.g., see https://en.wikipedia.org/wiki/Numerical_integration.

We focus on two basic types below. 

First, we consider geometric methods based upon rectangles. Then, we consider a stochastic form of integration called Monte Carlo integration. Monte Carlo methods are at the heart of many data science algorithms that rely upon some sort of stochastic implementation.

## Geometric (deterministic) methods
---

Many of the deterministic approaches to estimating integrals are based on partitioning $A$ (i.e., cutting up the set $A$ into a non-overlapping collection of subsets although we allow for shared boundaries between the subsets) such that the signed "areas" (or "volume" or generalizations of volume) between the graph of $f(x)$ and these subsets drawn on the horizontal axis (or plane or hyperplane) defined by the inputs are well-approximated by a simple geometric object for which we know the area (or volume or generalization of volume).

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Key Points:*** <span>

- The simplest such methods for 1-D problems involve the use of rectangles or trapezoids where the areas are easily computed from rules we learned in geometry.


- When $A$ is simply an interval, we typically choose the partition of $A$ to be equally sized sub-intervals and approximate the signed area of $f(x)$ over each subinterval using a rectangle that is as wide as the sub-interval with height given by the evaluation of $f(x)$ at some point in the sub-interval. 


- Adding up all the signed areas of rectangles gives an approximation to the integral. This is visualized below where we provide some options about where we can evaluate the function $f(x)$. 

<span style='background:rgba(0,255,255, 0.5); color:black'> Suggested activity:</span> Add a useful docstring to `compute_rect_rules` to describe the role of the various parameters. Add some doctests.

In [ ]:
def compute_rect_rules(f, a, b, n, rule='left'):
    '''
    Add a useful docstring here and doctest!
    '''
    ix = np.linspace(a, b, n+1)  # create the x-values to create the base of each rectangle 
    Delta_x = ix[1]-ix[0]  # compute the length of the base of each rectangle
    
    # Now determine the height of the rectangle by evaluating the
    # function f at some point along its base.
    if rule == 'left':
        iy = f(ix[0:-1])  # evaluate function at left-hand side of its base
    elif rule == 'right':
        iy = f(ix[1:])  # evaluate the function at right-hand side of its base
    else:
        iy = f(ix[0:-1]+0.5*Delta_x)  # evaluate the function at midpoint of its base
    
    # Now compute the approximate integral by adding up all 
    # the areas of the rectangles
    int_approx = np.sum(iy)*Delta_x
    
    return (ix, Delta_x, iy, int_approx)

***Let's do some numerical experiments and analysis.***

Below, we show how to

1. Use the `compute_rect_rules` function defined above with the default rule.

2. Analyze the rate of convergence (ROC).

3. Create interactive visualizations of the `compute_rect_rules` function that help us tell a story.

In [ ]:
f = lambda x: (x + 1) * (x - 5) * np.sin(3*x) 

In [ ]:
# This shows how to use the lambda function f above as well as the default rule
# to get an approximation
int_approx = compute_rect_rules(f, a=2, b=4, n=10)[3]

# Print the approximate integral
print(int_approx)

To study the ROC for any approximation process, we usually vary critical inputs over orders of magnitude and employ log-log plots to understand how errors are reduced by refinement of the inputs. 

In [ ]:
# The logspace function is an easy way to create an array that varies over various orders of magnitude
# The code below creates ns that go from 10**1 to 10**4
ns = np.logspace(1, 4, num=4) 
print(ns)

In [ ]:
print(ns.dtype)  # Unfortunately, the compute_rect_rules requires the number of rectangles to be of type int

In [ ]:
# An easy fix
ns = np.logspace(1, 4, num=4).astype('int')
print(ns)

In [ ]:
# Let's look at a list of approximations for increasing number of rectangles
int_approxes = []
for n in ns:
    int_approxes.append(compute_rect_rules(f, a=2, b=4, n=n)[3])
print(int_approxes)

In [ ]:
# Now compute the errors as a function of increasing number of rectangles
errors_left_approx = np.array(int_approxes) - quad(f, a=2, b=4)[0]
print(errors_left_approx)

In [ ]:
# Now visualize the convergence of the left-hand rule
plt.figure()
plt.loglog(ns, np.abs(errors_left_approx))
plt.xlabel('# of rects.', fontsize=12)
plt.title('Abs. Value of Errors vs. # of rects (left-hand rule)')

In [ ]:
# How to estimate the rate of convergence (ROC)?

# The above curve is approximately a straight line. We therefore fit a line
# of best fit below and report the slope of that line.

ROC_estimate = np.polyfit(np.log(ns), np.log(np.abs(errors_left_approx)), 1)[0]

# The slope of the line of best fit tells us how many orders of magnitude the error
# is decreased for every order of magnitude increase in the number of rectangles.
print(ROC_estimate) 

***Now we do some fancier visualization.***

In [ ]:
def plot_rectangle_rules(f, a, b, n, rule='left', x_min=0, x_max=1, fignum=0):
    # First get the approximation from a rectangle rule
    ix, Delta_x, iy, int_approx = compute_rect_rules(f, a, b, n, rule)
    
    # Get a more accurate approximation from quad
    # This is not necessary, but we use it for comparison sake
    int_ab = quad(f, a, b)[0] 
    
    ################# EVERYTHING BELOW IS JUST FOR PLOTTING
    x = np.linspace(x_min, x_max, 101)
    y = f(x)

    plt.figure(num=fignum)
    plt.clf()
    fig, ax = plt.subplots(figsize=(10,10), num=fignum)
    ax.plot(x, y, 'r', linewidth=2)

    plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis
    
    rects = []
    for i in range(n):
        rects.append(Rectangle((ix[i],0), Delta_x, iy[i]))    
    # Create patch collection with specified colour/alpha
    pc = PatchCollection(rects, facecolor='0.9', alpha=0.5,
                         edgecolor='0.5')
    ax.add_collection(pc)
   
    ax.set_title(r"$\int_a^b f(x)\mathrm{d}x = %3.2f \approx $ Sum of Rect. Areas = %3.2f" 
                 %(int_ab, int_approx), fontsize=20)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_xticks((a, b))
    ax.set_xticklabels(('$a$', '$b$'))
    plt.show()

In [ ]:
%reset -f out

%matplotlib widget
interact_manual(plot_rectangle_rules, 
         f = widgets.fixed(lambda x: (x + 1) * (x - 5) * np.sin(3*x)),
         a = widgets.FloatSlider(value=2, min=-1, max=5, step=0.1),
         b = widgets.FloatSlider(value=4, min=-1, max=5, step=0.1),
         n = widgets.IntSlider(value=3, min=1, max=50),
         rule=['left', 'right', 'middle'],
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(5),
         fignum=widgets.fixed(0));

***We visualize approximations of the integral for a different function below.***

In [ ]:
%reset -f out

%matplotlib widget
interact_manual(plot_rectangle_rules, 
         f = widgets.fixed(lambda x: x**2),
         a = widgets.FloatSlider(value=0, min=-1, max=5, step=0.1),
         b = widgets.FloatSlider(value=1, min=-1, max=5, step=0.1),
         n = widgets.IntSlider(value=10, min=1, max=1000),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(2),
         rule=['left', 'right', 'middle'],
         fignum=widgets.fixed(0));

***Let's make a `RectangleRule` `class` as a subclass of `Integration`.***

In [ ]:
class RectangleRule(Integration):
    '''
    
    '''
    def __init__(self, f, a=None, b=None, n=None, rule='Left'):
        '''
        
        '''
        super().__init__(f=f, a=a, b=b)
        self.n = n
        self.rule = rule
        
        return
        
    def change_rectangles(self, n=None, rule=None):
        '''
        
        '''
        if n is not None:
            self.n = n
        if rule is not None:
            self.rule = rule
            
        return
    
    def compute_integral_approx(self):
        '''
        
        '''
        self.ix = np.linspace(self.a, self.b, self.n+1)  # create the x-values to create the base of each rectangle 
        self.Delta_x = self.ix[1]-self.ix[0]  # compute the length of the base of each rectangle

        # Now determine the height of the rectangle by evaluating the
        # function f at some point along its base.
        if self.rule == 'left':
            self.iy = self.f(self.ix[0:-1])  # evaluate function at left-hand side of its base
        elif self.rule == 'right':
            self.iy = self.f(self.ix[1:])  # evaluate the function at right-hand side of its base
        else:
            self.iy = self.f(self.ix[0:-1]+0.5*self.Delta_x)  # evaluate the function at midpoint of its base

        # Now compute the approximate integral by adding up all 
        # the areas of the rectangles
        self.integral_approx = np.sum(self.iy)*self.Delta_x
        
        return self.integral_approx
    
    
    def plot_rectangle_approx(self, x_min=0, x_max=1, N=101, fignum=0):
        
        # Get a more accurate approximate for comparison sake using method
        # inherited from the super class and plot.
        self.evaluate_integral()

        ################# EVERYTHING BELOW IS JUST FOR PLOTTING
        x = np.linspace(x_min, x_max, N)
        y = self.f(x)

        plt.figure(num=fignum)
        plt.clf()
        fig, ax = plt.subplots(num=fignum)
        ax.plot(x, y, 'r', linewidth=2)
        
        plt.axhline(0, linewidth=1, linestyle=':', c='k')  # plot typical x-axis

        rects = []
        for i in range(self.n):
            rects.append(Rectangle((self.ix[i],0), self.Delta_x, self.iy[i]))    
        # Create patch collection with specified colour/alpha
        pc = PatchCollection(rects, facecolor='0.9', alpha=0.5,
                             edgecolor='0.5')
        ax.add_collection(pc)

        ax.set_title(r"$\int_a^b f(x)\mathrm{d}x = %3.2f \approx $ Sum of Rect. Areas = %3.2f" 
                     %(self.integral, self.integral_approx), fontsize=20)

        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xticks((self.a, self.b))
        ax.set_xticklabels(('$a$', '$b$'))
        plt.show()
        
        return
    
    def evaluate_and_plot_rectangle_approx(self, a, b, n, rule, x_min, x_max, N=101, fignum=0):
        '''
        
        '''
        self.change_rectangles(n=n, rule=rule)
        
        self.set_integral_limits(a=a, b=b)
        
        self.compute_integral_approx()
        
        self.plot_rectangle_approx(x_min=x_min, x_max=x_max, N=N, fignum=fignum)
        
        return

In [ ]:
f_rects = RectangleRule(f = lambda x: (x + 1) * (x - 5) * np.sin(3*x))

In [ ]:
%reset -f out

%matplotlib widget
interact_manual(f_rects.evaluate_and_plot_rectangle_approx, 
         a = widgets.FloatSlider(value=0, min=-1, max=5, step=0.1),
         b = widgets.FloatSlider(value=1, min=-1, max=5, step=0.1),
         n = widgets.IntSlider(value=10, min=1, max=1000),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(2),
         rule=['left', 'right', 'middle'],
         N = widgets.fixed(101),
         fignum=widgets.fixed(0));

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Rectangle rules</span><a id='activity-rectangle-rules'></a>

*For the sake of simplicity, this activity does not require the use of any classes.*

Feel free to create many new code and markdown cells below as you work through this activity.

- Create a wrapper function, `multiple_rect_approx`, which has all the same parameters as `compute_rect_rules` except that the `n` parameter is now assumed to be an *array* (or *list*) of integers containing different numbers of rectangles for which the approximate integrals should be computed. 

  This wrapper function should loop through all the values in the list of `n` values, call `compute_rect_rules` to obtain the associated approximate integral, and store all of the approximate integrals in a list that is returned by the wrapper function.

- For a number of different `lambda` functions including `f = lambda x: x`, `f = lambda x: x**2`, and at least one of your own choosing, compute the errors for the left-, right, and midpoint-rule approximations for `n=[10, 100, 1000]` for `a=0` and `b=1`. Store the errors as either a list or numpy array. 

    *Hint*: To compute the errors, you need to either know the exact value (or a very accurate approximation) of the integral of the function to compare to the rectangle rule approximations. Use the imported `quad` function from `scipy.integrate` as the "exact" value (it is a very good approximation in almost all cases).

- For each of your different `lambda` functions, create a log-log plot with a legend that shows the errors for each rectangle rule approximation vs. the number of rectangles. A log-log plot of errors is useful for analyzing the rate of convergence of a method. We like to see the errors *decrease* as we *increase* the computational effort. Estimate the rate of convergence using the `np.polyfit` function as shown in the example above.

   *Hint*: To use a log-log plot, everything must be non-negative, so you should use the `np.abs` function on the y-values of this plot as shown above.

- Comment on/interpret your results in the Markdown cell below. Some questions you should try to answer are the following. Which method appears to be most accurate/converge faster? Why do you think that is? *Hint:* Creating visualizations of the approximations of integrals with a small number of rectangles for the different methods may provide insight into how certain methods may be more likely to have "canceling" or "off-setting" errors within each rectangle. Some brief research online will also reveal the answer.

<hr style="border:5px solid cyan"> </hr>

## [Monte Carlo (MC) Integration](https://en.wikipedia.org/wiki/Monte_Carlo_integration) 
---

<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the first recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell with embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('uTBCgEurJm4', width=800, height=450)

The basic idea is to estimate $\int_A f(x)$ using the *average* of a random sample of $f(x)$ values. 

The question is this: how do we generate a random sample of $f(x)$ values to do this?

Well, we generate a random sample of $x$ values in the set $A$, and then evaluate the function $f$ at these random inputs. Then, we simply compute the sample average of $A$. 

Are there any sticky points to this?

Well, a few simple examples will illustrate just about all you possibly need to know.

Suppose $f(x)=5$ on $[0,1]$. From our previous example involving velocity and conceptualizing the integral of a real-valued function with a univariate input as the signed area between the curve and horizontal axis, we know that $\int_{[0,1]} 5 = 5(1-0)=5$. So, this is what our MC estimate should be approximating.

In fact, for any random sample of $x$-values between $[0,1]$, we will *always* get that the corresponding function values are *all* 5 because $f(x)$ is a constant 5 on this interval. So, what would the sample average be of these randomly sampled function values? Well, 5 of course! Whoa! The MC estimate is *exact* in this case. Great!

What if $f(x)=5$ on $[0,2]$? So, all we have done is change the set from $[0,1]$ to $[0,2]$. We know the exact answer should now be $\int_{[0,2]} 5 = 5(2-0)=10$. However, by the same reasoning, any random sample of $f(x)$ values will produce a sample average of $5$ not $10$. 

What if $f(x)=5$ on $[0,0.5]$? Again, the exact answer is $\int_{[0,0.5]} 5 = 5(0.5-2)=2.5$, yet any sample average of $f(x)$ values will produce $5$ not $2.5$. 

So, what is going on? Well, we need to *weight* the sample average by the *size* of the set $A$. 

In summary, if $\{x_i\}_{i=1}^N \subset A$ is a *uniform* random sample from the set $A$, and we let $\mu(A)$ denote the *measure* (i.e., size) of $A$, then an MC estimate of the integral is given by
$$
    \int_A f(x) \approx \mu(A)\frac{1}{N}\sum_{i=1}^N f(x_i).
$$

Let's play with this below and discuss what we are seeing.

<span style='background:rgba(0,255,255, 0.5); color:black'> Suggested activity:</span> Add a useful docstring to `compute_1D_MC_approx` to describe the role of the various parameters. Add some doctests.

In [ ]:
def compute_1D_MC_approx(f, a, b, n):
    x_random = np.random.uniform(low=a, high=b, size=int(n))
    mu_A = b-a  # mu_A=measure (size) of A
    avg_func = np.mean(f(x_random))
    MC_est = mu_A * avg_func
    return (avg_func, MC_est)

***Let's do some numerical experiments and analysis.***

Below, we show how to

1. Use the `compute_1D_MC_approx` function defined above and understand its variability due to random sampling.

2. Analyze the rate of convergence (ROC).

3. Create interactive visualizations of the `compute_1D_MC_approx` function that help us tell a story.

In [ ]:
f = lambda x: (x + 1) * (x - 5) * np.sin(3*x) 

In [ ]:
# These all look the same, but are different, why?
int_approx  = compute_1D_MC_approx(f, a=2, b=4, n=100)[1]
print(int_approx)
int_approx  = compute_1D_MC_approx(f, a=2, b=4, n=100)[1]
print(int_approx)
int_approx  = compute_1D_MC_approx(f, a=2, b=4, n=100)[1]
print(int_approx)

What we see above is an issue involving the random sampling of points between $[a,b]$ to generate an approximation of the average value of the function. As you can see by re-running the above code cell multiple times, there can be quite a bit of variability. Below, we compute 1000 approximations and visualize the results with a histogram.

In [ ]:
num_trials = 1000
int_approx = np.zeros(num_trials)
for i in range(num_trials):
    int_approx[i] = compute_1D_MC_approx(f, a=2, b=4, n=100)[1]

In [ ]:
plt.figure()
plt.hist(int_approx)
plt.axvline(quad(f, a=2, b=4)[0], color='k')  # Plot a vertical line where the "exact" value of the integral is

Looking at the histogram above, we see that the histogram appears to be centered around the exact value, so we compute the mean (sample average) of the estimates below.

In [ ]:
print(int_approx.mean())
print(quad(f, a=2, b=4)[0])

So, while any single MC estimate may be "bad", the expected value (i.e., the mean) of all MC estimates is quite good. In fact, the MC method produces what is known as an *unbiased* estimator of the integral meaning that its expected value is exactly the integral we want. Unfortunately, the variance in this estimator (i.e., how much any single MC estimate may vary around the estimate) can be quite large. How do we reduce the variance? We increase the number of points used to estimate the average value of the function. We show this below.

In [ ]:
# This can take a few seconds to run
# Students should comment each line of code here and explain the shape of int_approx and what 
# is stored in the ij component of this array.
num_trials = 1000
ns = np.logspace(2, 4, num=3).astype(int)
int_approx = np.zeros((3,num_trials))
for i in range(3):
    for j in range(num_trials):
        int_approx[i,j] = compute_1D_MC_approx(f, a=2, b=4, n=ns[i])[1]

In [ ]:
plt.figure(figsize=(8,4))
for i in range(3):
    plt.hist(int_approx[i,:], alpha=1-0.2*i, label='n=' + str(ns[i]))
plt.legend(fontsize=12)
plt.axvline(quad(f, a=2, b=4)[0], color='k') # Plot a vertical line where the "exact" value of the integral is

In the histograms above, we see that increasing `n` will reduce the variance in the MC estimates. In other words,  each individual MC estimate is more likely to be closer to its mean value. Since the mean values are all the same (the MC estimates are unbiased), this means that a larger `n` value is more likely to produce *accurate* estimates of the integral. 

It is more common to use the standard deviation to quantify variability in estimates around the mean value than the variance because it is in the same units as the mean value. The standard deviation is simply the square root of the variance. It can be computed using a built-in function within numpy as we show below where we analyze the rate of convergence (ROC) of MC estimates in terms of the standard deviation.

In [ ]:
# Now visualize the convergence of the left-hand rule
plt.figure()
plt.loglog(ns, int_approx.std(axis=1))
plt.xlabel('# of random samples', fontsize=12)
plt.title('St.Dev. of MC Estimates vs. # of samples')

In [ ]:
ROC_estimate = np.polyfit(np.log(ns), np.log(int_approx.std(axis=1)), 1)[0]

print(ROC_estimate) 

The above value is approximately -0.5, which is the theoretical rate guaranteed by the Central Limit Theorem (you should really take some probability theory and statistics classes is the lesson here). What this means is that for each order of magnitude increase in the number of samples, the standard deviation is decreased by *half* an order of magnitude.

Is this worse than the left-, right-, or mid-point rules for the deterministic methods? Yes, yes it is. Except there is a huge caveat here. The geometric methods for approximating integrals only work in extremely low dimensions. Geometric or other deterministic "quadrature" or "cubature" methods for approximating integrals suffer from what is known as the curse of dimensionality (look this up). We will not dwell on this here other than to say that as the dimension goes way up, the quality of the deterministic estimates goes way down unless an exponentially increasing number of points are used to estimate the integral.

The rate of convergence for the MC estimate is *independent* of dimension (although this does not quite tell the whole story, it is a big reason why random or pseudo-random approaches to estimating integrals are preferred in high-dimensions). 

Below, we create some visualizations to help capture the story above.

In [ ]:
def plot_MC_1D(f, a, b, n, x_min, x_max):
    avg_func, MC_est = compute_1D_MC_approx(f, a, b, n)
        
    ####### Everything below is for plotting
    x = np.linspace(x_min, x_max, 101)
    y = f(x)
    
    fig, ax = plt.subplots()
    ax.plot(x, y, 'r', linewidth=2)

    plt.axhline(0, linewidth=1, linestyle=':', c='k') #plot typical x-axis
    
    # Make the shaded region
    ix = np.linspace(a, b, 101)
    iy = f(ix)
    verts = [(a, 0), *zip(ix, iy), (b, 0)]
    poly = Polygon(verts, facecolor='0.9', edgecolor='0.5')
    ax.add_patch(poly)
    
    # Make an "average" rectangle corresponding to the MC_est
    verts = [(a, 0), (a, avg_func), (b, avg_func) , (b, 0)]
    rect = Polygon(verts, facecolor='b', alpha=0.25, edgecolor='0.5')
    ax.add_patch(rect)
    
    ax.set_title(r"$\int_a^b f(x)\mathrm{d}x \approx \frac{b-a}{N}\sum_{i=1}^N f(x_i) =$ %3.2f" %MC_est, fontsize=20)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_xticks((a, b))
    ax.set_xticklabels(('$a$', '$b$'))
    plt.show()

In [ ]:
%reset -f out

# Using an interact_manual widget so that we can re-run easily to observe
# the variations in the MC estimates due to random sampling.

interact_manual(plot_MC_1D, 
         f = widgets.fixed(lambda x: (x + 1) * (x - 5) * np.sin(3*x)),
         a = widgets.FloatSlider(value=1, min=-1, max=5, step=0.1),
         b = widgets.FloatSlider(value=2.5, min=-1, max=5, step=0.1),
         n = widgets.IntSlider(value=1E2, min=10, max=1E5, step=10),
         x_min = widgets.fixed(-1),
         x_max = widgets.fixed(5))

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Monte Carlo integration</span><a id='activity-MC'></a>

Feel free to create many new code and markdown cells below as you work through this activity.

- Complete the wrapper function, `multiple_1D_MC_approx`, in the code cell below except that we now assume `n` is a list of integers containing the different numbers of random points used to approximate the integral. The wrapper function also has an additional parameter `num_trials`, which is an `int` type that specifies the number of times (i.e., the number of trials) to repeat the approximations for each value in the list `n` values. The default `num_trials` is set to 10. 
    
  The function should output a 2-dimensional numpy array of shape `(len(n),num_trials)` where the `[i,j]` component (in Python syntax) gives the (j+1)th MC approximation of the integral using `n[i]` random points.  

In [ ]:
def multiple_1D_MC_approx(f, a, b, n, num_trials=10):
    
    int_f_approx = np.zeros(( , )) #COMPLETE THIS PART TO INITIALIZE THE ARRAY TO ALL ZEROS
        
    for i in range(len(n)):
        
        for j in range(num_trials):
            
            int_f_approx[i,j] = compute_1D_MC_approx() #COMPLETE THIS PART
            
    return int_f_approx

- For a number of different `lambda` functions including  `f = lambda x: x`, `f = lambda x: x**2`, and at least one of your own choosing, compute the errors for the MC approximations of the integrals with `n=[int(1E1), int(1E2), int(1E3), int(1E4)]`, `a=0`, and `b=1`. For each function, create a numpy array of shape `(len(n),num_trials)` to store the errors. 

- For each of your different `lambda` functions, create two separate log-log plots with a legend that shows the *means* and *variances* (over the number of trials) of the computed errors vs. the number of random points used in the MC approximations. For the means plot, you should compute the absolute values of the mean errors (not the mean of the absolute value) in order to get the plot to show correctly.

- Comment on your results in the Markdown cell below.

<hr style="border:5px solid cyan"> </hr>

## What if the set $A$ is complicated?

<span style='background:rgba(255,255,0, 0.25); color:black'> Run the code cell below and click the "play" button to see the first recorded lecture associated with this notebook.</span>

In [ ]:
# 1. Running this cell with embed the short recorded lecture associated with this part of the notebook
# 2. Press on the "play" button to start the video.

from IPython.display import YouTubeVideo

YouTubeVideo('PfIU7AK51Z8', width=800, height=450)

The typical built-in random number generators available in `numpy.random` or `scipy.stats` are great for generating random numbers/vectors in *nice* geometric sets like intervals in 1-D, rectangles in 2-D, rectangular boxes in 3-D, and generalizations of rectangular boxes in higher-dimensions. 

But, what if $A$ is not an interval or box-shaped? What if we can only describe $A$ in terms of relationships to other points or other geometric shapes? 

Well, we can do something referred to as accept-reject (sometimes just called rejection) sampling. 

At a conceptual level, imagine that we create a "dart board" out of a simple object (like a box) that *contains* the set $A$. The set $A$ is the "target" of the dart board. If we generate random numbers/vectors in the simple object (i.e., we "throw darts at the board") and *accept those that are inside of $A$* (i.e., we check if a "dart hits the target"), then we will have generated random samples within $A$. 

<span style='background:rgba(255,0,255, 0.25); color:black'> ***Key Points:*** <span>

- The proprtion of "darts" that fall in $A$ (i.e., the target) times the measure of the circumscribing box (i.e., the dart board size) gives an estimate of the measure of $A$ (which we denote by $\mu(A)$).
    
- We may have to throw a lot of darts to generate as many random samples in $A$ as we like.

Let's illustrate when $A$ is a unit disk in $\mathbb{R}^2$ that we fit into the dart board defined by the square $[-1,1]\times[-1,1]$.
    
<span style='background:rgba(0,255,255, 0.5); color:black'> Suggested activity:</span> Add a useful docstring to `random_disk_darts` to describe the role of the various parameters. Add some doctests.

In [ ]:
def random_disk_darts(n):
    #First throw darts at circumscribing square
    darts = np.random.uniform(low=-1, high=1, size=(int(n),2))
    
    #check which darts hit the target: a disk called "A"
    idx_in_A = []
    idx_not_in_A = []
    for i in range(n):
        if np.sqrt((darts[i,0]-0)**2 + (darts[i,1]-0)**2) < 1:
            idx_in_A.append(i)
        else:
            idx_not_in_A.append(i)
            
    mu_A_est = len(idx_in_A)/n * 4 # Estimates the area of A
            
    return darts, idx_in_A, idx_not_in_A, mu_A_est

<span style='background:rgba(0,255,255, 0.5); color:black'> Suggested activity:</span> Create a variant of the `random_disk_darts` function that uses a while-loop to generate `n` samples within $A$ and also returns the total `m` samples needed to generate these `n` samples. 

In [ ]:
def plot_random_disk_darts(n):
    darts, idx_in_A, idx_not_in_A, mu_A_est = random_disk_darts(n)
    
    ### Everything below is plotting
    fig, ax = plt.subplots(figsize=(10,10))
    
    ax.set_xlim([-1,1])
    ax.set_ylim([-1,1])
    ax.set_aspect(1)
    
    A = plt.Circle((0, 0), 1, color='r', alpha=0.1)
    ax.add_artist(A)

    ax.scatter(darts[idx_in_A,0], darts[idx_in_A,1], s=20, marker='x', c='k')    
    ax.scatter(darts[idx_not_in_A,0], darts[idx_not_in_A,1], s=20, marker='o', c='b')

    ax.set_title(r"$\mu(A)\approx \mu$(dart board)$\times$(percent of darts in $A$) $\approx$ %3.2f" 
                 %mu_A_est, fontsize=20)
    plt.show()

In [ ]:
%reset -f out

# Using an interact_manual widget so that we can re-run easily to observe
# the variations in the MC estimates of the area due to random sampling.

interact_manual(plot_random_disk_darts, 
                n = widgets.IntSlider(value=1E2, min=10, max=1E4, step=10))

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Integration over the dart board</span><a id='activity-MC-darts'></a>

- Based on the `random_disk_darts` function, create a new function, `my_MC_disk`, below that estimates the integral of a function $f$ over the unit disk. This function should have as inputs parameters `f` and `n` where `f` is the function to be integrated. It should return an estimate of the integral of `f` over the disk as well as the other outputs returned by `random_disk_darts`. 

  *Hint*: Two lines of code needed are:

 `avg_func = np.mean(f(darts[idx_in_A,:]))`

  and 

  `int_f_approx = mu_A_est * avg_func`

- Run some tests for different `lambda` functions of your choosing. 

- Based on the `plot_random_disk_darts` function, create a new function, `plot_MC_disk`, below that plots the darts that uses the `my_MC_disk` function to approximate the integral of a function `f` over the disk and plots the disk, darts that fall in/out of the disk, and displays the approximate integral of the function `f` as the title of the plot.

- Demonstrate that your `plot_random_disk_darts` function works on some `lambda` functions of your choosing below.

- ***Recommended but not required:*** Try to create new functions `my_MC_ellipse` and `plot_MC_ellipse_darts` that do similar MC computations and visualizations, but over ellipses instead of disks. Demonstrate that these functions work as expected. You may find reviewing the formula for a standard ellipse useful in doing this.

<hr style="border:5px solid cyan"> </hr>

<hr style="border:5px solid cyan"> </hr>

## <span style='background:rgba(0,255,255, 0.5); color:black'>Activity: Summary</span> <a id='activity-summary'/>

Summarize some of the key takeaways/points from this notebook in a list below and prepare a few code examples related to these takeaways/points in the code cells below. You need to have at least one example for each of your summary points and you need at least three summary points.

In this notebook, we have seen the following:

- [Your summary point 1 goes here]




- [Your summary point 2 goes here]




- [Your summary point 3 goes here]

<hr style="border:5px solid cyan"> </hr>

### <a href='#Contents'>Click here to return to Notebook Contents</a>